In [93]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import chainer
from chainer import training, Chain
from chainer.training import extensions
from chainer import links as L
from chainer import functions as F
from chainer.datasets import TupleDataset
from chainer import serializers

from chainer.functions.loss.mean_squared_error import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [113]:
df_dev = pd.read_csv('test.csv')


In [114]:
df_test = df_dev[['Pclass','Name','Sex','Age','SibSp','Parch','Fare','Embarked']]

In [116]:
def name_classifier(name_df):    
    name_class_df = pd.DataFrame(columns={'miss','mrs','master','mr'})
    
    for name in name_df:        
        if 'Miss' in name:
            df = pd.DataFrame([[1,0,0,0]],columns={'miss','mrs','master','mr'})
        elif 'Mrs' in name:
            df = pd.DataFrame([[0,1,0,0]],columns={'miss','mrs','master','mr'})
        elif 'Master' in name:
            df = pd.DataFrame([[0,0,1,0]],columns={'miss','mrs','master','mr'})
        elif 'Mr' in name:
            df = pd.DataFrame([[0,0,0,1]],columns={'miss','mrs','master','mr'})
        else :
            df = pd.DataFrame([[0,0,0,0]],columns={'miss','mrs','master','mr'})
        name_class_df = name_class_df.append(df,ignore_index=True)        
    return name_class_df

In [117]:
df_name = df_test['Name']
df_name_class = name_classifier(df_name)

In [118]:
df_test = pd.concat([df_test,df_name_class],axis =1)
df_test = df_test.drop('Name',axis = 1)
df_test['Age'] = df_test['Age'].fillna(df_test['Age'].mean())
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].mean())

In [119]:
for field in ['Sex','Embarked']:
    df_dummy = pd.get_dummies(df_test[field],prefix = field)
    df_test = pd.concat([df_test,df_dummy],axis  = 1)
    df_test = df_test.drop(field,axis = 1)

In [120]:
df_test.to_pickle('test.pkl')

In [101]:
df_test.isnull().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
miss          0
mrs           0
master        0
mr            0
dtype: int64

In [122]:
X = df_test.as_matrix().astype(np.float32)

In [121]:
df_test

,Pclass,Age,SibSp,Parch,Fare,miss,mrs,master,mr,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.50000,0,0,7.8292,0.0,0.0,0.0,1.0,0,1,0,1,0
1,3,47.00000,1,0,7.0000,0.0,1.0,0.0,0.0,1,0,0,0,1
2,2,62.00000,0,0,9.6875,0.0,0.0,0.0,1.0,0,1,0,1,0
3,3,27.00000,0,0,8.6625,0.0,0.0,0.0,1.0,0,1,0,0,1
4,3,22.00000,1,1,12.2875,0.0,1.0,0.0,0.0,1,0,0,0,1
5,3,14.00000,0,0,9.2250,0.0,0.0,0.0,1.0,0,1,0,0,1
6,3,30.00000,0,0,7.6292,1.0,0.0,0.0,0.0,1,0,0,1,0
7,2,26.00000,1,1,29.0000,0.0,0.0,0.0,1.0,0,1,0,0,1
8,3,18.00000,0,0,7.2292,0.0,1.0,0.0,0.0,1,0,1,0,0
9,3,21.00000,2,0,24.1500,0.0,0.0,0.0,1.0,0,1,0,0,1


In [123]:
class MyChain(Chain):
    def __init__(self):
        super(MyChain,self).__init__(
            l1 = L.Linear(None, 10),
            l2 = L.Linear(None, 6),
            l3 = L.Linear(None,2),
            bn1 = L.BatchNormalization(size = 14),
            bn2 = L.BatchNormalization(size = 10),
            bn3 = L.BatchNormalization(size = 6)
        )
        
    def __call__(self, x, train = True):
        h = F.dropout(self.bn1(x),ratio = 0.2, train=train)
        h = F.dropout(F.relu(self.bn2(self.l1(x))), train = train)
        h = F.dropout(F.relu(self.bn3(self.l2(h))), train = train)
        y = self.l3(h)
        
        return y

In [124]:
df_pred = pd.DataFrame(index=df_test.index, columns=[])

In [125]:
n_fold = 10

In [126]:
for i in range(n_fold):
    model = L.Classifier(MyChain())
    serializers.load_npz('model/model_'+str(i)+'.npz',model)
    pred = F.softmax(model.predictor(X,train=False)).data[:,1]
    df_pred['pred'+str(i)]=pred

In [127]:
df_test['prob'] = df_pred.mean(1)

In [128]:
df_test

,Pclass,Age,SibSp,Parch,Fare,miss,mrs,master,mr,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,prob
0,3,34.50000,0,0,7.8292,0.0,0.0,0.0,1.0,0,1,0,1,0,0.107192
1,3,47.00000,1,0,7.0000,0.0,1.0,0.0,0.0,1,0,0,0,1,0.333957
2,2,62.00000,0,0,9.6875,0.0,0.0,0.0,1.0,0,1,0,1,0,0.110495
3,3,27.00000,0,0,8.6625,0.0,0.0,0.0,1.0,0,1,0,0,1,0.132951
4,3,22.00000,1,1,12.2875,0.0,1.0,0.0,0.0,1,0,0,0,1,0.434639
5,3,14.00000,0,0,9.2250,0.0,0.0,0.0,1.0,0,1,0,0,1,0.146014
6,3,30.00000,0,0,7.6292,1.0,0.0,0.0,0.0,1,0,0,1,0,0.589391
7,2,26.00000,1,1,29.0000,0.0,0.0,0.0,1.0,0,1,0,0,1,0.129442
8,3,18.00000,0,0,7.2292,0.0,1.0,0.0,0.0,1,0,1,0,0,0.840594
9,3,21.00000,2,0,24.1500,0.0,0.0,0.0,1.0,0,1,0,0,1,0.104608


In [129]:
X = df_test.as_matrix().astype(np.float32)

In [130]:
clf = joblib.load('model/Stage01/LogisticRegression.pkl') 

In [131]:
y_pred = clf.predict(X)

In [132]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [133]:
df_prob = df_dev[['PassengerId']]

In [134]:
df_prob['Survived']=y_pred

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [135]:
df_prob.to_csv('submit/ensemble.csv',index=False)

In [137]:
df_prob

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
